In [2]:
!pip uninstall tensorflow
!pip install tensorflow==1.15
import tensorflow as tf
print(f">> Tensorflow Version: {tf.__version__}")

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 512kB 56.0MB/s 
     |████████████████████████████████| 3.8MB 52.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=1d12810bf86706a09d85222bcfc224d868fbeea61e11e350259ad243a11b8806
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probabilit

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
%cd /content/drive/My Drive/dl_assignment_04
tf.__version__

/content/drive/My Drive/dl_assignment_04


'1.15.0'

## Preset별로 Training이 일어나는 부분

- `presets` 리스트에 훈련시킬 프리셋 번호를 입력하면 된다.
- 프리셋들은 `presets_and_results.py`에 선언되어 있음.

In [2]:
%cd /content/drive/My Drive/dl_assignment_04
import tensorflow as tf
import os
import time
import datetime
from tensorflow.keras.datasets.cifar10 import load_data
import data_helpers as dh
from resnet import ResNet

import presets_and_results as pr
import math


# presets = [1, 2, 3]
presets = [5]
isColab = True
for preset in presets:
    p = pr.Preset(preset)
    pr.Preset().del_all_flags(tf.app.flags.FLAGS)
    tf.reset_default_graph()
    if not isColab:
        # Model Hyperparameters
        tf.flags.DEFINE_float("lr", p.lr, "learning rate (default=0.1)")
        tf.flags.DEFINE_float("lr_decay", p.lr_decay, "learning rate decay rate(default=0.1)")
        tf.flags.DEFINE_float("l2_reg_lambda", p.l2_reg_lambda, "L2 regularization lambda (default: 0.0)")
        tf.flags.DEFINE_float("relu_leakiness", p.relu_leakiness, "relu leakiness (default: 0.1)")
        tf.flags.DEFINE_integer("num_residual_units", p.num_residual_units, "The number of residual_units (default: 5)")
        tf.flags.DEFINE_integer("num_classes", p.num_classes, "The number of classes (default: 10)")

        # Training parameters
        tf.flags.DEFINE_integer("batch_size", p.batch_size, "Batch Size (default: 64)")
        tf.flags.DEFINE_integer("num_epochs", p.num_epochs, "Number of training epochs (default: 200)")
        tf.flags.DEFINE_integer("evaluate_every", p.evaluate_every, "Evaluate model on dev set after this many steps (default: 100)")
        tf.flags.DEFINE_integer("checkpoint_every", p.checkpoint_every, "Save model after this many steps (default: 100)")
        tf.flags.DEFINE_integer("num_checkpoints", p.num_checkpoints, "Number of checkpoints to store (default: 5)")
        tf.flags.DEFINE_boolean("data_augmentation", p.data_augmentation, "data augmentation option")

        # Misc Parameters
        tf.flags.DEFINE_boolean("allow_soft_placement", p.allow_soft_placement, "Allow device soft device placement")
        tf.flags.DEFINE_boolean("log_device_placement", p.log_device_placement, "Log placement of ops on devices")
        FLAGS = tf.flags.FLAGS

    elif isColab:
        # Model Hyperparameters
        tf.app.flags.DEFINE_float("lr", p.lr, "learning rate (default=0.1)")
        tf.app.flags.DEFINE_float("lr_decay", p.lr_decay, "learning rate decay rate(default=0.1)")
        tf.app.flags.DEFINE_float("l2_reg_lambda", p.l2_reg_lambda, "L2 regularization lambda (default: 0.0)")
        tf.app.flags.DEFINE_float("relu_leakiness", p.relu_leakiness, "relu leakiness (default: 0.1)")
        tf.app.flags.DEFINE_integer("num_residual_units", p.num_residual_units, "The number of residual_units (default: 5)")
        tf.app.flags.DEFINE_integer("num_classes", p.num_classes, "The number of classes (default: 10)")

        # Training parameters
        tf.app.flags.DEFINE_integer("batch_size", p.batch_size, "Batch Size (default: 64)")
        tf.app.flags.DEFINE_integer("num_epochs", p.num_epochs, "Number of training epochs (default: 200)")
        tf.app.flags.DEFINE_integer("evaluate_every", p.evaluate_every, "Evaluate model on dev set after this many steps (default: 100)")
        tf.app.flags.DEFINE_integer("checkpoint_every", p.checkpoint_every, "Save model after this many steps (default: 100)")
        tf.app.flags.DEFINE_integer("num_checkpoints", p.num_checkpoints, "Number of checkpoints to store (default: 5)")
        tf.app.flags.DEFINE_boolean("data_augmentation", p.data_augmentation, "data augmentation option")
        tf.app.flags.DEFINE_string('f', '', 'kernel')

        # Misc Parameters
        FLAGS = tf.app.flags.FLAGS

    (x_train_val, y_train_val), (x_test, y_test) = load_data()
    x_train, y_train, x_test, y_test, x_val, y_val = dh.shuffle_data(x_train_val, y_train_val, x_test, y_test, FLAGS.num_classes)

    with tf.Graph().as_default():
        if not isColab:
            session_conf = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement, log_device_placement=FLAGS.log_device_placement)
            sess = tf.Session(config=session_conf)
        elif isColab:
            sess = tf.Session()
        with sess.as_default():
            resnet = ResNet(FLAGS) #ResNet 클래스의 인스턴스 생성 후 Hyperparameter가 정의돼 있는 FLAGS로 초기화

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            decayed_lr = tf.train.exponential_decay(FLAGS.lr, global_step, 24000, FLAGS.lr_decay, staircase=True)
            optimizer = tf.train.MomentumOptimizer(learning_rate=decayed_lr, momentum=0.9)
            grads_and_vars = optimizer.compute_gradients(resnet.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
            train_ops = [train_op] + resnet.extra_train_ops
            train_ops = tf.group(*train_ops)

            # Output directory for models and summaries
            p.timestamp = int(time.time())
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", str(p.timestamp)))
            print("> Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", resnet.loss)
            acc_summary = tf.summary.scalar("accuracy", resnet.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {resnet.X: x_batch, resnet.Y: y_batch}
                _, step, lr, summaries, loss, accuracy = sess.run(
                    [train_ops, global_step, decayed_lr, train_summary_op, resnet.loss, resnet.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().strftime("%m/%d %H:%M:%S")
                prog = FLAGS.num_epochs * math.ceil(45000/FLAGS.batch_size)
                print(f"- [{time_str}] Preset={p.preset} Tstmp={p.timestamp}  Step=({format(step, '05')}/{format(prog, '05')}) Loss={format(loss, '<8g')} Acc={format(accuracy, '<9g')}")
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {resnet.X: x_batch, resnet.Y: y_batch}
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, resnet.loss, resnet.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().strftime("%m/%d %H:%M:%S")
                prog = FLAGS.num_epochs * math.ceil(45000/FLAGS.batch_size)
                print(f"- [{time_str}] Preset={p.preset} Tstmp={p.timestamp}  Step=({format(step, '05')}/{format(prog, '05')}) Loss={format(loss, '<8g')} Acc={format(accuracy, '<9g')}")
                if writer:
                    writer.add_summary(summaries, step)
                return accuracy

            # Generate batches
            if FLAGS.data_augmentation:  # data augmentation 적용시
                batches = dh.batch_iter_aug(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
            else:
                batches = dh.batch_iter(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            max = 0
            start_time = time.time()
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    accuracy = dev_step(x_val, y_val, writer=dev_summary_writer)
                    print("")
                    if accuracy > max:
                        max = accuracy
                        path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                        early_stopping_epoch = current_step
                        print("Saved model checkpoint to {}\n".format(path))
                        # 결과 처리 위한 부분
                        p.val_accuracy = accuracy
                        p.early_stopping_epoch = early_stopping_epoch

                training_time = (time.time() - start_time) / 60
                p.training_time = training_time
            p.saveCurPresetToPickle()
            p.saveCurTrainResultToTxt()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
- [10/05 19:52:39] Preset=5 Tstmp=1601900363  Step=(65600/70400) Loss=0.692158 Acc=0.859375 

Evaluation:
- [10/05 19:52:42] Preset=5 Tstmp=1601900363  Step=(65600/70400) Loss=0.791315 Acc=0.8446   

- [10/05 19:52:43] Preset=5 Tstmp=1601900363  Step=(65601/70400) Loss=0.51403  Acc=0.9375   
- [10/05 19:52:43] Preset=5 Tstmp=1601900363  Step=(65602/70400) Loss=0.902944 Acc=0.8125   
- [10/05 19:52:43] Preset=5 Tstmp=1601900363  Step=(65603/70400) Loss=0.73481  Acc=0.859375 
- [10/05 19:52:44] Preset=5 Tstmp=1601900363  Step=(65604/70400) Loss=0.833364 Acc=0.820312 
- [10/05 19:52:44] Preset=5 Tstmp=1601900363  Step=(65605/70400) Loss=0.646773 Acc=0.875    
- [10/05 19:52:44] Preset=5 Tstmp=1601900363  Step=(65606/70400) Loss=0.731346 Acc=0.851562 
- [10/05 19:52:45] Preset=5 Tstmp=1601900363  Step=(65607/70400) Loss=0.744449 Acc=0.875    
- [10/05 19:52:45] Preset=5 Tstmp=1601900363  Step=(65608/70400) Loss=0.809642 Acc=0.820312 
- [10/05 19:52:46] P

In [5]:
%cd /content/drive/My Drive/dl_assignment_04

import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

import os
import presets_and_results as pr
import pickle

pickles = pr.Preset().readAllPickles()
for pckl in pickles:
        if not pckl.finished:
            (x_train_val, y_train_val), (x_test, y_test) = load_data()

            # Eval Parameters
            pr.Preset().del_all_flags(tf.flags.FLAGS)
            tf.app.flags.DEFINE_string("checkpoint_dir", f"./runs/{str(pckl.timestamp)}/checkpoints", "Checkpoint directory from training run")
            tf.app.flags.DEFINE_string('f', '', 'kernel')

            FLAGS = tf.app.flags.FLAGS
            # ==================================================
            y_test_one_hot = np.eye(10)[y_test]
            y_test_one_hot = np.squeeze(y_test_one_hot, axis=1)
            x_test = (x_test/127.5) - 1

            checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
            graph = tf.Graph()
            with graph.as_default():
                sess = tf.Session()
                with sess.as_default():
                    saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
                    saver.restore(sess, checkpoint_file)

                    # Get the placeholders from the graph by name
                    X = graph.get_operation_by_name("X").outputs[0]
                    Y = graph.get_operation_by_name("Y").outputs[0]
                    accuracy = graph.get_operation_by_name("accuracy/accuracy").outputs[0]

                    test_accuracy = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
                    pckl.test_accuracy = test_accuracy
                    print('- Preset           :', pckl.preset)
                    print('- Timestamp        :', pckl.test_accuracy)
                    print('- Test Max Accuracy:', test_accuracy)
                    pckl.finished = True
       
            pckl.saveCurTrainResultToTxt(name="eval_overall.txt")
            pckl.saveCurPresetToPickle()


/content/drive/My Drive/dl_assignment_04
[<presets_and_results.Preset object at 0x7f8f131399e8>, <presets_and_results.Preset object at 0x7f8f13139978>, <presets_and_results.Preset object at 0x7f8f13139b00>, <presets_and_results.Preset object at 0x7f8f13139be0>, <presets_and_results.Preset object at 0x7f8f13139ba8>]
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/dl_assignment_04/runs/1601900363/checkpoints/model-61000
- Preset           : 5
- Timestamp        : 0.8635
- Test Max Accuracy: 0.8635
> Preset 5번의 학습기록이 ./INFOS/eval_overall.txt에 기록되었습니다.
> Preset 5번의 학습정보가 ./INFOS/pickles/5_1601900363.pickle에 저장되었습니다.


In [ ]:
# 한꺼번에 텐서보드를 불러오기 위한 파일
import os

def makeDir(_filename):
    return os.path.join(os.path.curdir, 'runs', _filename)

filenames = os.listdir(os.path.join(os.path.curdir, 'runs'))

# 사용하려는 명령어는 아래와 같은 꼴
# tensorboard --logdir=name1:/path/to/logs/1,name2:/path/to/logs/2
command_names = []
for filename in filenames:
    name = f'{filename}'
    ddir = f'{makeDir(filename)}'
    name += ':' + ddir
    command_names.append(name)

command = f'tensorboard --logdir='
command += ','.join(command_names)
command += ' --host localhost'

os.system(f'{command}')

2

In [ ]:
import scipy
scipy.__version__


'1.1.0'

In [ ]:
%cd /content/drive/My Drive/dl_assignment_04
import tensorflow as tf
import data_helpers as dh
import numpy as np
import presets_and_results as pr


# 이부분에 사용할 모델의 Timestamp를 적으면 추론 시작
whatModelTimestamp = input("- 추론에 사용할 모델의 타임스탬프를 입력하세요.")

# Eval Parameters
pr.Preset().del_all_flags(tf.app.flags.FLAGS)
tf.app.flags.DEFINE_string('f', '', 'kernel')


tf.app.flags.DEFINE_string("checkpoint_dir", f"./runs/{whatModelTimestamp}/checkpoints", "Checkpoint directory from training run")
FLAGS = tf.flags.FLAGS
dataset = dh.read_my_images("./example/*")

checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    with sess.as_default():
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        X = graph.get_operation_by_name("X").outputs[0]
        Y = graph.get_operation_by_name("Y").outputs[0]
        predictions = graph.get_operation_by_name("logit/predictions").outputs[0]

        inference_result = sess.run(predictions, feed_dict={X: dataset})
        llist = "airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck".split(', ')
        for i in inference_result:
            print(llist[i])


/content/drive/My Drive/dl_assignment_04
- 추론에 사용할 모델의 타임스탬프를 입력하세요.1601870034
./example/bori.jpg
./example/bori__sleep.jpg
./example/airplane.jpg
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/dl_assignment_04/runs/1601870034/checkpoints/model-61600
automobile
cat
airplane
